# $\texttt{feyntrop}$ tutorial: A two loop 3-point Feynman graph

After $\texttt{feyntrop}$ has been properly compiled in the working directory (see README.md), we can use it in any python environment. The first step is to import the $\texttt{feyntrop}$ module, assuming $\texttt{feyntrop.so}$ and $\texttt{py}$_$\texttt{feyntrop.py}$ are in the working directory, we can run:

In [10]:
from py_feyntrop import *

## The graph
* We specify the Feynman graph via a list of weighted edges.
* Each item of the list is given in the format $((u,w),\nu)$, where $u$ and $w$ are the vertices that the edge connects and $\nu$ is the associated edge weights.
* We use 0-indexing. So, the vertices are numbered $0,1,2,\ldots$.
* External vertices must come first, so $V_\text{ext} = \{0,1,2\}$ label external partcles, and $V_\text{int} = \{3\}$.

The graph is then given as an edge list:

In [11]:
graph = [((0,1), 1), ((1,3), 1), ((3,2), 1), ((2,0), 1), ((0,3), 1)]

## Kinematics
* For each edge in the list we have to give an associated squared mass.
* For simplicity we set $m_e^2 = 0.2$ for all $e$.

In [12]:
masses_sqr = [0.2] * len(graph)

* Next we provide values for the momentum configuration.
* The values for squared momenta $p_0^2, \ldots, p_{|V_\text{ext}|-2}^2$ are denoted by tuples $(\texttt{p}$_$\texttt{sqr}[v], \text{value})$. In this case $|V_\text{ext}| = 3$.
* The values for Mandelstam variables $s_{ij}=(p_i+p_j)^2$ with $0 \leq i < j \leq |V_\text{ext}|-2$ are written as $(\texttt{s}[i,j], \text{value})$.

In [13]:
momentum_vars = [(p_sqr[0], 0), (p_sqr[1], 0), (s[0,1], 1)]

## Additional settings
* $\texttt{D0}$ is the integer part of the spacetime dimension $D = D_0 - 2\epsilon$.
* We expand up to but not including $\texttt{eps}$_$\texttt{order}$.
* $\texttt{Lambda}$ denotes the deformation parameter $\lambda$.
* $\texttt{N}$ is the number of Monte Carlo sampling points.

In [15]:
D0 = 2
eps_order = 5
Lambda = 7.6
N = int(1e7)

## Tropical integration
* $\texttt{trop}$_$\texttt{res}$ is the value of the Feynman integral, without any prefactor.
* $\texttt{Itr}$ is the normalization factor in the tropical measure.

In [ ]:
trop_res, Itr = tropical_integration(graph, masses_sqr, momentum_vars, D0, eps_order, Lambda, N)

Prefactor: gamma(2*eps + 3).
Generalized permutahedron property: fulfilled.
(Effective) kinematic regime: Minkowski
Analytic continuation: activated. Lambda = 7.6


* This yields the $\epsilon$-expansion with prefactor $\frac{\Gamma(\omega)}{\Gamma(\nu_1) \cdots \Gamma(\nu_E)}$ included, where $\omega$ is the superficial degree of divergence and $E$ is the number of edges.

In [11]:
eps_expansion(trop_res, graph, D0)

174.171297*I - 93.20022824 + eps*(-720.2473552 + 544.0741932*I) + eps**2*(-2113.294178 + 497.2079595*I) + eps**3*(-3568.151747 - 673.8189532*I) + eps**4*(-3868.691529 - 2718.146343*I) + O(eps**5)